In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib
import gc
import itertools
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestClassifier
import zipfile
import os

import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from tqdm import tqdm_notebook
from tqdm import notebook
from itertools import product
import xgboost as xgb

%matplotlib inline

In [2]:
zf = zipfile.ZipFile('competitive-data-science-predict-future-sales.zip') 

In [3]:
# Read items
items = pd.read_csv(zf.open('items.csv'))

# Read sales train
sales_train = pd.read_csv(zf.open('sales_train.csv'))
sales_train['date'] = pd.to_datetime(sales_train['date'], format= '%d.%m.%Y')
sales_train['revenue'] = sales_train['item_price']*sales_train['item_cnt_day']

# Read item categories
item_categories_translated = pd.read_csv('dimensions_df/item_categories_translated.csv')

# Read shops
shops_translated = pd.read_csv('dimensions_df/shops_translated.csv')

# Read test
test = pd.read_csv(zf.open('test.csv'))


In [4]:
# Let's order this by date to spot some additional trends
sales_train_ordered = sales_train.sort_values('date').copy()

In [5]:

shops_translated['city_en'] = shops_translated['city_en'].astype('category')
shops_translated['city_id'] = shops_translated['city_en'].cat.codes


In [6]:
columns = ['sub_category_1_en', 'sub_category_2_en']

for column in columns:
    item_categories_translated[column] = item_categories_translated[column].astype('category')
    item_categories_translated[column.replace('en', 'id')] = item_categories_translated[column].cat.codes

In [7]:
# We do not need all columns, so we will use just column ids and columns in English
shop_mask = ['shop_id', 'city_en', 'city_id']
category_mask = ['item_category_id', 'sub_category_1_en', 'sub_category_2_en',
                 'sub_category_1_id', 'sub_category_2_id']


shops_translated = shops_translated[shop_mask]
item_categories_translated = item_categories_translated[category_mask]

In [8]:
items = items[['item_id', 'item_category_id']]
items =pd.merge(items, item_categories_translated, on = 'item_category_id', how = 'left')

In [9]:
sales_train_ordered = pd.merge(sales_train_ordered, shops_translated, on = 'shop_id', how = 'left')
sales_train_ordered = pd.merge(sales_train_ordered, items, on = 'item_id', how = 'left')

In [10]:
sales_train_ordered['month'] = sales_train_ordered['date'].dt.month
sales_train_ordered['year'] = sales_train_ordered['date'].dt.year

sales_train_ordered.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,city_en,city_id,item_category_id,sub_category_1_en,sub_category_2_en,sub_category_1_id,sub_category_2_id,month,year
0,2013-01-01,0,18,5823,2500.0,1.0,2500.0,Krasnoyarsk,8,35,payment cards,PSN,19,38,1,2013
1,2013-01-01,0,27,5573,849.0,1.0,849.0,Moscow,10,2,accessories,PS3,16,36,1,2013
2,2013-01-01,0,7,1006,399.0,1.0,399.0,Voronezh,26,67,Presents,Development,12,19,1,2013
3,2013-01-01,0,19,17707,899.0,1.0,899.0,Kursk,9,19,Games,PS3,3,36,1,2013
4,2013-01-01,0,14,19548,149.0,1.0,149.0,Kazan,5,40,Movie,DVD,5,18,1,2013


In [11]:
sales_train_ordered['item_cnt_month'] = sales_train_ordered.groupby('date_block_num')['item_cnt_day'].transform('sum')

In [12]:
sales_train_ordered

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,city_en,city_id,item_category_id,sub_category_1_en,sub_category_2_en,sub_category_1_id,sub_category_2_id,month,year,item_cnt_month
0,2013-01-01,0,18,5823,2500.0,1.0,2500.0,Krasnoyarsk,8,35,payment cards,PSN,19,38,1,2013,131479.0
1,2013-01-01,0,27,5573,849.0,1.0,849.0,Moscow,10,2,accessories,PS3,16,36,1,2013,131479.0
2,2013-01-01,0,7,1006,399.0,1.0,399.0,Voronezh,26,67,Presents,Development,12,19,1,2013,131479.0
3,2013-01-01,0,19,17707,899.0,1.0,899.0,Kursk,9,19,Games,PS3,3,36,1,2013,131479.0
4,2013-01-01,0,14,19548,149.0,1.0,149.0,Kazan,5,40,Movie,DVD,5,18,1,2013,131479.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,2015-10-31,33,41,21386,169.0,1.0,169.0,Rostov-on-Don,15,40,Movie,DVD,5,18,10,2015,71056.0
2935845,2015-10-31,33,21,988,199.0,1.0,199.0,Moscow,10,37,Movie,Blu,5,7,10,2015,71056.0
2935846,2015-10-31,33,41,21377,169.0,1.0,169.0,Rostov-on-Don,15,40,Movie,DVD,5,18,10,2015,71056.0
2935847,2015-10-31,33,22,10207,1199.0,1.0,1199.0,Moscow,10,30,PC Games,standard edition,10,57,10,2015,71056.0
